## Imports

In [1]:
import pypsa
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os


c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


## Read network file

In [17]:
# Function to read in the network file

def read_network_file():
    network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc'
    n = pypsa.Network(network)
    return n

network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc'
n = pypsa.Network(network)



c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

## Scaling factors and Decomissioning

The scaling factors are used to increase the demand and decrease the emission constraint. The decomissioning (and comissioning, if applicable) is added on a yearly basis as well.

In [18]:
# Demand scaling factor - based on the initial demand in 2021

scale_demand_old = {
    2022: 1.039555791,
    2023: 1.037440228,
    2024: 1.030106894,
    2025: 1.029322163,
    2026: 1.051886792,
    2027: 1.044491339,
    2028: 1.044448186,
    2029: 1.044491013,
    2030: 1.044447874,
    2031: 1.044477281,
    2032: 1.044422438,
    2033: 1.040027091,
    2034: 1.039984371,
    2035: 1.040012523,
    'back':0.569269673, # scaling factor for scaling from 2035 to 2021
    'direct' : 1.7566367 # scaling factor from 2021 to 2035
}


scale_demand = {
    2024: 1.058723565,
    2025: 1.060816836,
    2026: 1.033372562,
    2027: 1.027658017,
    2028: 1.032454682,
    2029: 1.030974469,
    2030: 1.030564438,
    2031: 1.029585799,
    2032: 1.029366414,
    2033: 1.048001634,
    2034: 1.041580000,
    2035: 1.041580000,
    2036: 1.041580000,
    2037: 1.041580000,
    'back':0.58467212, # scaling factor for scaling from 2035 to 2021
    'direct' : 1.7566367 # scaling factor from 2021 to 2035
}


In [19]:
scale_cost_sudden = {
    ('OCGT', 2024): 1.0,
    ('OCGT', 2025): 1.0,
    ('OCGT', 2026): 0.24,
    ('OCGT', 2027): 1.0,
    ('OCGT', 2028): 1.0,
    ('OCGT', 2029): 1.0,
    ('OCGT', 2030): 1.0,
    ('OCGT', 2031): 1.0,
    ('OCGT', 2032): 1.0,
    ('OCGT', 2033): 1.0,
    ('OCGT', 2034): 1.0,
    ('OCGT', 2035): 1.0,
    ('OCGT', 2036): 1.0,
    ('OCGT', 2037): 1.0,

    ('CCGT', 2024): 1.0,
    ('CCGT', 2025): 1.0,
    ('CCGT', 2026): 0.27,
    ('CCGT', 2027): 1.0,
    ('CCGT', 2028): 1.0,
    ('CCGT', 2029): 1.0,
    ('CCGT', 2030): 1.0,
    ('CCGT', 2031): 1.0,
    ('CCGT', 2032): 1.0,
    ('CCGT', 2033): 1.0,
    ('CCGT', 2034): 1.0,
    ('CCGT', 2035): 1.0,
    ('CCGT', 2036): 1.0,
    ('CCGT', 2037): 1.0,

    ('oil', 2024): 1.0,
    ('oil', 2025): 1.0,
    ('oil', 2026): 0.29,
    ('oil', 2027): 1.0,
    ('oil', 2028): 1.0,
    ('oil', 2029): 1.0,
    ('oil', 2030): 1.0,
    ('oil', 2031): 1.0,
    ('oil', 2032): 1.0,
    ('oil', 2033): 1.0,
    ('oil', 2034): 1.0,
    ('oil', 2035): 1.0,
    ('oil', 2036): 1.0,
    ('oil', 2037): 1.0,
}

In [6]:
scale_cost_gradual = {
    ('OCGT', 2024): 0.82,
    ('OCGT', 2025): 0.85,
    ('OCGT', 2026): 0.87,
    ('OCGT', 2027): 0.88,
    ('OCGT', 2028): 0.89,
    ('OCGT', 2029): 0.90,
    ('OCGT', 2030): 0.91,
    ('OCGT', 2031): 0.92,
    ('OCGT', 2032): 0.93,
    ('OCGT', 2033): 0.93,
    ('OCGT', 2034): 0.94,
    ('OCGT', 2035): 0.94,
    ('OCGT', 2036): 0.94,
    ('OCGT', 2037): 0.95,

    ('CCGT', 2024): 0.84,
    ('CCGT', 2025): 0.86,
    ('CCGT', 2026): 0.88,
    ('CCGT', 2027): 0.89,
    ('CCGT', 2028): 0.90,
    ('CCGT', 2029): 0.91,
    ('CCGT', 2030): 0.92,
    ('CCGT', 2031): 0.92,
    ('CCGT', 2032): 0.93,
    ('CCGT', 2033): 0.93,
    ('CCGT', 2034): 0.94,
    ('CCGT', 2035): 0.94,
    ('CCGT', 2036): 0.94,
    ('CCGT', 2037): 0.95,

    ('oil', 2024): 0.854,
    ('oil', 2025): 0.873,
    ('oil', 2026): 0.887,
    ('oil', 2027): 0.898,
    ('oil', 2028): 0.908,
    ('oil', 2029): 0.916,
    ('oil', 2030): 0.922,
    ('oil', 2031): 0.928,
    ('oil', 2032): 0.933,
    ('oil', 2033): 0.937,
    ('oil', 2034): 0.941,
    ('oil', 2035): 0.944,
    ('oil', 2036): 0.947,
    ('oil', 2037): 0.950,
}

In [20]:
# emissions decline

start_year = 2023
start_value = 2728743
end_year = 2050
end_value = 0

years = range(start_year, end_year + 1)
emissions = []

decrease_per_year = (start_value - end_value) / (end_year - start_year)

for i in years:
    emission = max(start_value - decrease_per_year * (i - start_year), end_value)
    emissions.append(emission)

# Make into dict
index = np.arange(2023,2051)
emission_limit = dict(zip(index, emissions))
emission_limit


{2023: 2728743.0,
 2024: 2627678.4444444445,
 2025: 2526613.888888889,
 2026: 2425549.3333333335,
 2027: 2324484.777777778,
 2028: 2223420.222222222,
 2029: 2122355.6666666665,
 2030: 2021291.111111111,
 2031: 1920226.5555555555,
 2032: 1819162.0,
 2033: 1718097.4444444445,
 2034: 1617032.8888888888,
 2035: 1515968.3333333333,
 2036: 1414903.7777777778,
 2037: 1313839.222222222,
 2038: 1212774.6666666665,
 2039: 1111710.111111111,
 2040: 1010645.5555555555,
 2041: 909581.0,
 2042: 808516.4444444443,
 2043: 707451.8888888888,
 2044: 606387.333333333,
 2045: 505322.7777777775,
 2046: 404258.222222222,
 2047: 303193.6666666665,
 2048: 202129.111111111,
 2049: 101064.5555555555,
 2050: 0.0}

In [5]:
# decomissioning of powerplants

decom = {
2022 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.3, 'BO 2 OCGT': 36.76, 'BO 3 oil':0},
2023 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 36.77, 'BO 3 oil':0},
2024 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2025 : {'BO 0 OCGT':25.44, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2026 : {'BO 0 OCGT':0, 'BO 1 OCGT':17.86, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2027 : {'BO 0 OCGT':0, 'BO 1 OCGT':105.87, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2028 : {'BO 0 OCGT':0, 'BO 1 OCGT':16.9, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2029 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2030 : {'BO 0 OCGT':0, 'BO 1 OCGT':97.85, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2031 : {'BO 0 OCGT':0, 'BO 1 OCGT':109.37, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2032 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2033 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':2.16},
2034 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2035 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
}

# To force expansion, this dicts is used
expansion = {
    2022 : {'BO 0 geothermal':5, 'BO 0 solar':13, 'BO 2 ror':126},
    2023: { 'BO 2 solar':13, 'BO 3 ror':126},
    2024: {'BO 1 biomass': 21, 'BO 1 onwind':53, 'BO 0 solar':13, 'BO 2 ror':126},
    2025: { 'BO 1 onwind':24,'BO 2 solar':13, 'BO 3 ror':126},
    2026: {'BO 3 biomass': 21, 'BO 0 solar':13, 'BO 2 ror':126},
    2027: {'BO 0 geothermal':45, 'BO 1 onwind':25,'BO 2 solar':13, 'BO 3 ror':126},
    2028: {'BO 1 biomass': 21, 'BO 1 onwind':25, 'BO 0 solar':13, 'BO 2 ror':126},
    2029: {'BO 2 solar':13, 'BO 3 ror':126},
    2030: {'BO 3 biomass': 21, 'BO 1 onwind':35, 'BO 0 solar':13, 'BO 2 ror':126},
    2031: {},
    2032:{},
    2033:{},
    2034:{},
    2035:{},
}


# Nodes in the model (can change!!!!)
#0 : Southern
# 1: Oriental
#2: Central
#3: Northern

In [11]:
n.generators

#
# OCGT -> CCGT

,p_nom_max,weight,p_nom,capital_cost,efficiency,p_nom_min,marginal_cost,bus,carrier,control,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,
10 onwind,1.554366e+03,10.011657,0.00,109295.569252,1.0,0.0,0.015,10,onwind,Slack,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
10 solar,2.686871e+03,481.564185,0.00,50855.958267,1.0,0.0,0.010,10,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
21 onwind,2.955995e+04,490.343900,0.00,109295.569252,1.0,0.0,0.015,21,onwind,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
21 ror,inf,1.890000,1.89,270940.715283,0.9,0.0,0.000,21,ror,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
21 solar,4.209422e+04,7880.946130,0.00,50855.958267,1.0,0.0,0.010,21,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73 solar,4.489479e+03,757.116362,0.00,50855.958267,1.0,0.0,0.010,73,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
89 onwind,2.569871e+03,128.600148,0.00,109295.569252,1.0,0.0,0.015,89,onwind,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
89 solar,4.001768e+03,790.759928,0.00,50855.958267,1.0,0.0,0.010,89,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0


## All changes

This cell makes all changes that need to be done to run the next year all at once and saves it in a new file. 

In [27]:
# Function that implements all yearly changes

def yearly_changes(n,year):
    print(year)
    # ------- DEMAND ---------
    upscaling_factor = scale_demand[year]
    n.loads_t.p_set = n.loads_t.p_set * upscaling_factor
    display(round((n.loads_t.p_set.sum().sum()/1000000),4))

    # ------- EMISSIONS -------
    n.global_constraints.constant = emission_limit[year]
    display(n.global_constraints.constant)

    #-------- COSTS ---------
    indexes = {}
    for car in ['OCGT', 'CCGT', 'oil']:
        mask = n.generators['carrier'] == car
        indexes[car] = n.generators.index[mask].tolist()
    for car in indexes:
        n.generators.loc[indexes[car], 'marginal_cost'] = n.generators.loc[indexes[car], 'marginal_cost'] / scale_cost_sudden[(car, year)]

    # ------- GENERATOR EXTENSTION -----
    solved_network = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/{scen_folder}/{scen}_{year-1}.nc'
    m = pypsa.Network(solved_network)
    additional_exp= m.generators.p_nom_opt - m.generators.p_nom
    # replace negative values with 0
    for index, value in additional_exp.items():
        if value < 0:
            additional_exp[index]=0
    # add expansion to previous network
    n.generators.p_nom = n.generators.p_nom.add(additional_exp, fill_value=0)
    n.generators.p_nom_min = n.generators.p_nom_min.add(additional_exp, fill_value=0)
    #display(n.generators.p_nom)


    # ------- STORES ----------
    additional_stores = m.stores.e_nom_opt - m.stores.e_nom
    additional_stores
    n.stores.e_nom = n.stores.e_nom.add(additional_stores, fill_value = 0)
    n.stores.e_nom_min = n.stores.e_nom_min.add(additional_stores, fill_value = 0)
    #display(n.stores.e_nom.sum())


    # ------- LINKS -------
    additional_links = m.links.p_nom_opt - m.links.p_nom
    additional_links
    n.links.p_nom = n.links.p_nom.add(additional_links, fill_value = 0)
    n.links.p_nom_min = n.links.p_nom_min.add(additional_links, fill_value = 0)
    #display(n.links.p_nom.sum())


    # ------- DECOM ---------
    #for index,value in decom[year].items():
    #    n.generators.loc[index, 'p_nom'] = n.generators.loc[index].p_nom - value
    #    n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min - value

    # -------- EXPANSION ----------
    # for index,value in expansion[year].items():
    #     n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min + value

    # ------- HYDRO-DECOM-------- # is done seperately, because the  code is different
    #if year == 2032:
    #    n.storage_units.loc['BO 2 hydro','p_nom'] = n.storage_units.loc['BO 2 hydro'].p_nom - 2.55

    # ------- SAVING ----------
    n.export_to_netcdf('C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc')

## Snakemake process

In [28]:
# Function that renames the results file and moves it to the scenario folder

def rename_results_file(year):
    current_location = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/results/networks/'
    old_filename = "elec_s_all_ec_lcopt_Co2L-1H.nc"
    new_filename = f"{scen}_{year}.nc"
    new_location = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/{scen_folder}/'

    # Create the full paths for the old and new files
    old_file_path = os.path.join(current_location, old_filename)
    new_file_path = os.path.join(new_location, new_filename)

    # Rename the file
    os.rename(old_file_path, new_file_path)


In [30]:
# determine scenario
scen_folder = 'results_cost_sudden_cap'
scen = 'cost_s_cap'

# !!!! Make sure the upscaling, emission constraints and decomissioning is run!!!

# run for loop over all years
for year in range(2023, 2038):
    n = read_network_file()
    if year == 2023:
        !snakemake -j 8 solve_all_networks
        rename_results_file(year)
    else:
        yearly_changes(n,year)
        !snakemake -j 8 solve_all_networks 
        rename_results_file(year)  

# And you're DONE!!! :D


c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
No parameters matching '_test' found


C:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(
Building DAG of jobs...
Nothing to be done (all requested files are present and up to date).
Complete log: .snakemake\log\2023-11-30T204851.070205.snakemake.log
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future err

2024


11.2141

...

14.6871

GlobalConstraint
CO2Limit    1819162.0
Name: constant, dtype: float64

c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

^C


FileNotFoundError: [WinError 2] Systemet finner ikke angitt fil: 'C:/Users/noraky/Documents/Test_4/pypsa-earth/results/networks/elec_s_all_ec_lcopt_Co2L-1H.nc' -> 'C:/Users/noraky/Documents/Test_4/pypsa-earth/results_cost_sudden_cap/cost_s_cap_2032.nc'

# Changes to the Network

Here all the general changes (except for expandabels, that's at the end of this file) that can be made to the model are shown. If more than the two last rules of the model are run, some of these changes may have to be done again. Use the "Checks" section to check your current network file. 

### Add Biomass in Node 0 and 3

The nodes may need to be adapted, depending on your model

In [ ]:
n.add("Generator",
            'BO 0 biomass',
            bus='BO 0',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)


n.add("Generator",
            'BO 2 biomass',
            bus='BO 2',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)

### Add ROR 

Nodes may vary

In [ ]:
n.storage_units

In [ ]:
n.add("Generator",
            'BO 2 ror',
            bus='BO 2',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

n.add("Generator",
            'BO 3 ror',
            bus='BO 3',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

### Biomas Max Capacity

The nodes may vary, the larger biomass max capacity is located in the Oriental zone

In [ ]:
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'biomass':
        if index == 'BO 1 biomass':
        # Update the p_max_pu value to 0.64
            n.generators.at[index, 'p_nom_max'] = 357.13
        else:
            n.generators.at[index, 'p_nom_max'] = 21.01

n.generators.p_nom_max

### Battery max capacity

In [ ]:
# discharger capacity

for index, row in n.links.iterrows():
    if row['carrier'] == 'battery discharger':
        n.links.at[index, 'p_nom_max'] = 225

display(n.links.p_nom_max)


# storage capacity

for index, row in n.stores.iterrows():
    if row['carrier'] == 'battery':
        n.stores.at[index, 'e_nom_max'] = 1350

n.stores.e_nom_max

### Capacity factors

With the new addition to model where the annual capacity factors can be set, this is not necessary anymore.

In [25]:
# Check the current status
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       0.64
OCGT       0.64
biomass    0.32
oil        1.00
onwind     1.00
ror        0.50
solar      1.00
Name: p_max_pu, dtype: float64

In [18]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'OCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 # 0.08 #0.64
    if row['carrier'] == 'CCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 #0.45 #0.64

    if row['carrier'] == 'biomass':
        # Update the p_max_pu value to 0.72
        n.generators.at[index, 'p_max_pu'] = 0.32 #0.72 for the future 
    if row['carrier'] == 'ror':
        n.generators.at[index, 'p_max_pu'] = 0.5

        
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       0.64
OCGT       0.64
biomass    0.32
oil        1.00
onwind     1.00
ror        0.50
solar      1.00
Name: p_max_pu, dtype: float64

# Save elec_s_4_ec_lcopt_Co2L-1H file

If you made changes to network file at the beginning, they need to be saved again. This code can be used for that. 

In [14]:
n.export_to_netcdf('C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc')

INFO:pypsa.io:Exported network elec_s_all_ec_lcopt_Co2L-1H.nc has stores, loads, carriers, links, buses, global_constraints, lines, storage_units, generators


<xarray.Dataset>
Dimensions:                               (snapshots: 8760,
                                           investment_periods: 0, stores_i: 82,
                                           loads_i: 82, loads_t_p_set_i: 82,
                                           carriers_i: 11, links_i: 164,
                                           buses_i: 164,
                                           global_constraints_i: 1,
                                           lines_i: 93, storage_units_i: 12,
                                           storage_units_t_inflow_i: 12,
                                           generators_i: 252,
                                           generators_t_p_max_pu_i: 164)
Coordinates: (12/14)
  * snapshots                             (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                    (investment_periods) int64 
  * stores_i                              (stores_i) object '0 battery' ... '...
  * loads_i                               (loads_i) object '0' '1' ... '103'
  * loads_t_p_set_i                       (loads_t_p_set_i) object '0' ... '103'
  * carriers_i                            (carriers_i) object 'CCGT' ... 'bat...
    ...                                    ...
  * global_constraints_i                  (global_constraints_i) object 'CO2L...
  * lines_i                               (lines_i) object '124795784_0' ... ...
  * storage_units_i                       (storage_units_i) object '0 hydro' ...
  * storage_units_t_inflow_i              (storage_units_t_inflow_i) object '...
  * generators_i                          (generators_i) object 'C29.0' ... '...
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object '0...
Data variables: (12/93)
    snapshots_snapshot                    (snapshots) datetime64[ns] 2013-01-...
    snapshots_objective                   (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 1.0 1.0 ... 1.0
    investment_periods_objective          (investment_periods) float64 
    investment_periods_years              (investment_periods) float64 
    ...                                    ...
    generators_lifetime                   (generators_i) float64 50.0 ... inf
    generators_p_nom_max                  (generators_i) float64 inf inf ... inf
    generators_weight                     (generators_i) float64 nan ... 1.97
    generators_control                    (generators_i) object 'Slack' ... 'PQ'
    generators_p_max_pu                   (generators_i) float64 0.64 ... 0.5
    generators_t_p_max_pu                 (snapshots, generators_t_p_max_pu_i) float64 ...
Attributes:
    network_name:           PyPSA-Eur
    network_pypsa_version:  0.24.0
    network_srid:           4326
    meta:                   {"version": "0.2.2", "tutorial": false, "logging"...

# Checks

If you don't know the setting of your current network file, use the following cells to check it. 

In [5]:
# Which generators do exist?

n.generators

,carrier,bus,p_nom_min,p_nom,p_nom_extendable,efficiency,marginal_cost,capital_cost,build_year,lifetime,...,sign,marginal_cost_quadratic,committable,start_up_cost,shut_down_cost,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
C29,OCGT,93,17.861572,17.861572,True,0.35,15.685714,47234.561404,1975,50.0,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
C30,OCGT,93,16.902920,16.902920,True,0.35,15.685714,47234.561404,1977,50.0,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
C31,OCGT,93,19.131431,19.131431,True,0.35,15.685714,47234.561404,1980,50.0,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
C32,OCGT,93,19.860825,19.860825,True,0.35,15.685714,47234.561404,1988,50.0,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
C33,OCGT,93,97.854365,97.854365,True,0.35,15.685714,47234.561404,1999,30.0,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21 hydro,ror,70,0.000000,7.540000,False,0.90,0.000000,270940.715283,0,inf,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
22 hydro,ror,55,0.000000,11.490000,False,0.90,0.000000,270940.715283,0,inf,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0
23 hydro,ror,55,0.000000,5.150000,False,0.90,0.000000,270940.715283,0,inf,...,1.0,0.0,False,0.0,0.0,NaN,NaN,1.0,1.0,0.0


In [38]:
n.generators.to_csv('C:/Users/noraky/Documents/Test_4/pypsa-earth/generators.csv')

In [71]:
# Which generators are currently extendable?

n.generators.p_nom_extendable


Generator
10 onwind     True
10 solar      True
21 onwind     True
21 ror       False
21 solar      True
             ...  
73 solar      True
89 onwind     True
89 solar      True
9 onwind      True
9 solar       True
Name: p_nom_extendable, Length: 71, dtype: bool

In [31]:
n.generators.p_nom_extendable.to_csv('C:/Users/noraky/Documents/Test_4/pypsa-earth/generators_extendable.csv')

In [31]:
# Check the current status of the cp
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       1.0
OCGT       1.0
biomass    1.0
oil        1.0
onwind     1.0
ror        1.0
solar      1.0
Name: p_max_pu, dtype: float64

In [50]:
# Which generators have which expansion limit?

n.generators.p_nom_max

Generator
10 onwind    1.554366e+03
10 solar     2.686871e+03
21 onwind    2.955995e+04
21 ror                inf
21 solar     4.209422e+04
                 ...     
73 solar     4.489479e+03
89 onwind    2.569871e+03
89 solar     4.001768e+03
9 onwind     1.913721e+03
9 solar      6.916290e+02
Name: p_nom_max, Length: 71, dtype: float64

In [51]:
# Which links have which expansion limit?

n.links.p_nom_max

Link
3 battery charger        inf
5 battery charger        inf
6 battery charger        inf
9 battery charger        inf
10 battery charger       inf
21 battery charger       inf
27 battery charger       inf
29 battery charger       inf
32 battery charger       inf
34 battery charger       inf
42 battery charger       inf
46 battery charger       inf
48 battery charger       inf
49 battery charger       inf
52 battery charger       inf
55 battery charger       inf
58 battery charger       inf
60 battery charger       inf
61 battery charger       inf
63 battery charger       inf
64 battery charger       inf
65 battery charger       inf
68 battery charger       inf
69 battery charger       inf
72 battery charger       inf
73 battery charger       inf
89 battery charger       inf
3 battery discharger     inf
5 battery discharger     inf
6 battery discharger     inf
9 battery discharger     inf
10 battery discharger    inf
21 battery discharger    inf
27 battery discharger    inf
29 batter

In [95]:
# What is the sum of the demand in the network?

round(n.loads_t.p_set.sum().sum()/1000000,8)

10.59208457

In [53]:
# Is there any installed storage capacity?

n.stores.e_nom

Store
3 battery     0.0
5 battery     0.0
6 battery     0.0
9 battery     0.0
10 battery    0.0
21 battery    0.0
27 battery    0.0
29 battery    0.0
32 battery    0.0
34 battery    0.0
42 battery    0.0
46 battery    0.0
48 battery    0.0
49 battery    0.0
52 battery    0.0
55 battery    0.0
58 battery    0.0
60 battery    0.0
61 battery    0.0
63 battery    0.0
64 battery    0.0
65 battery    0.0
68 battery    0.0
69 battery    0.0
72 battery    0.0
73 battery    0.0
89 battery    0.0
Name: e_nom, dtype: float64

In [18]:
# What is the current emission limit?

n.global_constraints.constant.loc['CO2Limit']

77500000000.0

In [19]:
# Which stores have a maximum expansion limit?

n.stores.e_nom_max

Store
3 battery     inf
5 battery     inf
6 battery     inf
9 battery     inf
10 battery    inf
21 battery    inf
27 battery    inf
29 battery    inf
32 battery    inf
34 battery    inf
42 battery    inf
46 battery    inf
48 battery    inf
49 battery    inf
52 battery    inf
55 battery    inf
58 battery    inf
60 battery    inf
61 battery    inf
63 battery    inf
64 battery    inf
65 battery    inf
68 battery    inf
69 battery    inf
72 battery    inf
73 battery    inf
89 battery    inf
Name: e_nom_max, dtype: float64

# Restoring to 2021

When you reached your final year, you can easily restore your network to the year 2021. It is recommended, though, to save the initial 2021 file at the beginning. 

## Demand

In [12]:
year = 'back'
upscaling_factor = scale_demand[year]   # 0.569269674



n.loads_t.p_set = n.loads_t.p_set * upscaling_factor

year = 2023

In [26]:
n.loads_t.p_set.sum().sum()

10592084.469436923

## Emission limit

In [ ]:
n.global_constraints.constant = 1000000000 #emission_limit[2021]
n.global_constraints.constant

## Expansions

In [ ]:
solved_network = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/results_NS/NS_2021.nc'
m = pypsa.Network(solved_network)

n.generators.p_nom = m.generators.p_nom
n.generators.p_nom

In [ ]:
n.generators.p_nom_min = m.generators.p_nom_min
n.generators.p_nom_min

In [ ]:
# new biomass to 0
for index, row in n.generators.iterrows():
    if index == 'BO 0 biomass':
        n.generators.at[index, 'p_nom'] = 0
    if index == 'BO 2 biomass':
            n.generators.at[index, 'p_nom'] = 0
n.generators.p_nom

In [ ]:
n.stores.e_nom = 0

n.stores.e_nom.sum()

In [ ]:
n.links.p_nom = 0
n.links.p_nom.sum()

# Changing the expandabel carriers

This can also be done in the config file, but doing it here saves you from running the model again and have to do all the other changes again as well.

### RES to TRUE

In [ ]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'biomass':
        
        n.generators.at[index, 'p_nom_extendable'] = True 
    if row['carrier'] == 'solar':
        
        n.generators.at[index, 'p_nom_extendable'] = True 

    if row['carrier'] == 'onwind':
       
        n.generators.at[index, 'p_nom_extendable'] = True 

### Fossils to FALSE

In [ ]:
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'OCGT':
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'CCGT':
        
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'oil':
       
        n.generators.at[index, 'p_nom_extendable'] = False 

n.generators.p_nom_extendable